In [1]:
# from main_control import Servo, Camera, Robot, BatteryLevel
from console_control import _controller, Controller
from utils import aug_view, aug_road, FPS

import time
import cv2

serial Open!


In [2]:
control = Controller()

import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model('models/custom_model.tf', custom_objects={'tf':tf})
print('loaded')
model.summary()
input_data = np.zeros((1, 96, 96, 1), dtype=np.float32)
model(input_data)
print('predicted')

loaded
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 1)]       0         
_________________________________________________________________
dropout (Dropout)            (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        1312      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
average_pooling2d (AveragePo (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)      

In [8]:
def stream_n_move(img):
    global pred_left, pred_right
    cv2.imwrite('main.jpg', img)
    try:
        img = cv2.rectangle(img, pt1=(0,      int(224/2)), pt2=(20,  int((224/2) + ((224/2)*pred_left))),  color=(33, 22, 132), thickness=-1)
        img = cv2.rectangle(img, pt1=(224-20, int(224/2)), pt2=(224, int((224/2) + ((224/2)*pred_right))), color=(33, 22, 132), thickness=-1)
    except NameError: pass
    return img

In [5]:
import threading
stopper = False
first_pred = True

def control_thread():
    global stopper, first_pred, control, pred_left, pred_right
    while not stopper:
        img = control.camera.get_image(dtype='array')
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img.astype(np.float32)/255, axis=0)
        
        if first_pred: 
            print('First prediction starting')
        
        pred_left, pred_right = model(img, training=False).numpy()[0]
            
        if first_pred: 
            first_pred = False
            print('First prediction finished')
        
        pred_left, pred_right = float(pred_left), float(pred_right)
        
#         control.robot.custom_steer(-pred_left, -pred_right, 0.7, 
#                                                1.0, 1.0, 1.0, 1.0)
        time.sleep(0.05)
    control.robot.stop()
    
thread = threading.Thread(target=control_thread)
thread.start()

First prediction starting
First prediction finished


In [9]:
control.servo.goto(pitch=-45)
control.camera.start_stream(stream_n_move)
display(control.camera.stream)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [7]:
stopper = True

In [ ]:
control.robot.stop()